In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from pandas.core.algorithms import mode
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils.np_utils import to_categorical
from keras.layers import Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
import cv2
from sklearn.model_selection import train_test_split
import pickle
import os
import pandas as pd
import random
from keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
from google.colab.patches import cv2_imshow

drivePath = 'drive/MyDrive/AI/detect_face/' #put your link here
path = Path(drivePath)
print(path)

In [ ]:
lableFile = Path('drive/MyDrive/AI/name.csv') #file excel label all names of class

In [ ]:
count = 0
images = []
classNo = []
myList = os.listdir(path)  #need to modify this code
noOfClasses = len(myList)
print("Load classes ....")
for x in range (0,len(myList)):
    myPicList = os.listdir(str(path) + "/" + str(count))
    for y in myPicList:
        curImg = cv2.imread(str(path)+"/"+str(count)+"/"+y)
        curImg = cv2.resize(curImg,(32,32))
        images.append(curImg)
        classNo.append(count)
    print(count, end=" ")
    count +=1
print(" ")
images = np.array(images)
classNo = np.array(classNo)

In [ ]:
batch_size_val = 4
step_per_epoch_val = 50
epochs_val=  4
imageDimensions = (32,32,3)
testRatio = 0.2
validationRatio = 0.2

x_train, x_test, y_train, y_test = train_test_split(images,classNo,test_size = testRatio)
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size = validationRatio)
print("Data Shapes")
print("Train",end="");print(x_train.shape,y_train.shape)

#data = pd.read_csv(lableFile, sep='delimiter', header=None)
data = pd.read_csv(lableFile)
print("data shape",data.shape,type(data))

num_of_samples = []
cols = 5
num_classes = noOfClasses
fig,axs = plt.subplots(nrows=num_classes, ncols=cols, figsize=(10,100))
fig.tight_layout()
for i in range(cols):
    for j,row in data.iterrows():
        x_selected = x_train[y_train == j]
        axs[j][i].imshow(x_selected[random.randint(0,len(x_selected)-1), :, :], cmap=plt.get_cmap("gray"))
        axs[j][i].axis("off")
        if i ==2:
            axs[j][i].set_title(str(j)+ "-"+row["Name"])
            num_of_samples.append(len(x_selected))

print(num_of_samples)
plt.figure(figsize=(12,4))
plt.bar(range(0,num_classes),num_of_samples)
plt.title("Distribution of trainning data")
plt.xlabel("class number")
plt.ylabel("Number of image")
plt.show()


In [ ]:
def grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

def equalize(img):
    img = cv2.equalizeHist(img)
    return img
    
def preprocessing(img):
    img = grayscale(img)
    img = equalize(img)
    img = img/255
    return img


x_train = np.array(list(map(preprocessing,x_train)))
x_validation = np.array(list(map(preprocessing,x_validation)))
x_test = np.array(list(map(preprocessing,x_test)))
#cv2.imshow("Grayscale images ", x_train[random.randint(0,len(x_train)-1)])

x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2],1)
x_validation=x_validation.reshape(x_validation.shape[0],x_validation.shape[1],x_validation.shape[2],1)
x_test=x_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],1)

dataGen = ImageDataGenerator(width_shift_range=0.1,
                                height_shift_range =0.1,
                                zoom_range=0.2,
                                shear_range=0.1,
                                rotation_range=10)

dataGen.fit(x_train)
batches = dataGen.flow(x_train,y_train,batch_size=6)
x_batch, y_batch = next(batches)

fig,axs = plt.subplots(1,5,figsize=(20,5))
fig.tight_layout()
for i in range(5):
    axs[i].imshow(x_batch[i].reshape(imageDimensions[0],imageDimensions[1]))
    axs[i].axis('off')
plt.show()

y_train =  to_categorical(y_train,noOfClasses)
y_validation = to_categorical(y_validation,noOfClasses)
y_test = to_categorical(y_test,noOfClasses)

In [ ]:
def myModel():
    no_Of_Filters = 60
    size_of_Filter = (5,5)
    size_Of_Filter2=(3,3)
    size_of_pool=(2,2)
    no_Of_Nodes = 500
    model= Sequential()
    model.add(Conv2D(32,(3,3),activation='relu',kernel_initializer='he_uniform',padding='same', input_shape=(32,32,1)))
    model.add(Conv2D(32,(3,3),activation='relu',kernel_initializer='he_uniform',padding='same'))
  #model.add(MaxPooling2D((2,2)))


    model.add(Conv2D(64,(3,3),activation='relu',kernel_initializer='he_uniform',padding='same'))
    #model.add(Conv2D(64,(3,3),activation='relu',kernel_initializer='he_uniform',padding='same'))
    model.add(MaxPooling2D((2,2)))

    model.add(Conv2D(128,(2,2),activation='relu',kernel_initializer='he_uniform',padding='same'))
    #model.add(Conv2D(128,(2,2),activation='relu',kernel_initializer='he_uniform',padding='same'))
    model.add(MaxPooling2D((2,2)))


    model.add(Flatten())
    model.add(Dense(256,activation='relu',kernel_initializer='he_uniform'))
    model.add(Dense(3,activation='softmax'))
    #model.compile(keras.optimizers.Adam(learning_rate = 0.001),loss='categorical crossentropy',metrics=['accuracy'])
    model.compile(
        optimizer = 'adam',
        loss = 'categorical_crossentropy',
        metrics=['accuracy']
    )
    return model
model = myModel()
print(model.summary())

In [ ]:

#history = model.fit(dataGen.flow(x_train,y_train,batch_size = batch_size_val),epochs = epochs_val,validation_data=(x_test,y_test))
history = model.fit(x_train,y_train,epochs = epochs_val,validation_data=(x_test,y_test))
plt.figure(1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training','validation'])
plt.title('loss')
plt.xlabel('epoch')
plt.figure(2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training','validation'])
plt.title('Accuracy')
plt.xlabel('epoch')
plt.show()

model.save('ankhangdang.h5')

In [ ]:
from tensorflow.keras.models import load_model
model2= load_model('ankhangdang.h5')

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array 
from keras.models import load_model
import numpy as np  

In [ ]:
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(32, 32,1))	
	
	return img
labels = {
    0: 'Dang',
    1: 'An',
    2: 'Khang'
}
filename = 'an.jpg'
img = load_image(filename)
plt.imshow(img)
img = img_to_array(img)
print('shape is: ',img.shape)
img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img = img.reshape(1,32,32,1) 
img.shape
result = model.predict(img)
print(result)
np.argmax(result) 
print('This is: ',labels[np.argmax(result)]) 

In [ ]:
#end